<a href="https://colab.research.google.com/github/Darally06/Labo_arbol/blob/main/Copia_de_LAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATORIO 1:
### *ESTRUCTURA DE DATOS 2*
Por: Juan Aguirre, Daniela Guerra, Maria Gabriela Rey.

Marzo 17, 2024


### 1. Librerias

In [ ]:
from google.colab import files
from collections import deque
import os
import shutil
import re
from PIL import Image
from typing import Any, List, Optional, Tuple
from graphviz import Digraph
from IPython.display import Image
from ast import NodeTransformer

### 2. Manejo de las imágenes


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Guardar nombres de imagenes
def obtener_categoria(name):
    try:
        categorias = {
            "dog": "dogs",
            "rider": "humans",
            "horse": "horses",
            "cat": "cats",
            "bike": "bikes",
            "car": "cars"
        }
        for key, value in categorias.items():
            if key in name.lower():
                return value
        return "flowers"
    except Exception as e:
        print(f"Error al obtener la categoría: {e}")
        return None

def organizar_carpetas(destino, carpetas):
    try:
        # Crear carpeta general
        if not os.path.exists(destino):
            os.makedirs(destino)

        # Categorías
        for categoria in carpetas:
            carpeta = os.path.join(destino, categoria)
            if not os.path.exists(carpeta):
                os.makedirs(carpeta)

        archivos = os.listdir(destino)
        for archivo in archivos:
            ruta = os.path.join(destino, archivo)
            if os.path.isfile(ruta):
                categoria = obtener_categoria(archivo)
                c_destino = os.path.join(destino, categoria)
                shutil.move(ruta, os.path.join(c_destino, archivo))
    except Exception as e:
        print(f"Error al organizar las carpetas: {e}")

def generar_ruta(ruta_archivo, nombre):
    ruta = os.path.join(ruta_archivo, nombre)
    return ruta

# Procesar imagenes
def procesar_imagenes(destino, carpetas):
    try:
        imagenes = {}
        for categoria in carpetas:
            carpeta = os.path.join(destino, categoria)
            imagenes[categoria] = []
            archivos = os.listdir(carpeta)
            for archivo in archivos:
                ruta_archivo = os.path.join(carpeta, archivo)
                if os.path.isfile(ruta_archivo):
                    name = os.path.basename(ruta_archivo)
                    peso_archivo = os.path.getsize(ruta_archivo) / (1024 * 1024)
                    ruta = os.path.join(ruta_archivo, name)
                    registro = {"Nombre": name, "Categoria": categoria, "Peso": peso_archivo, "Ruta": ruta}
                    imagenes[categoria].append(registro)
        return imagenes
    except Exception as e:
        print(f"Error al procesar las imágenes: {e}")
        return {}

### 4. Crear Arbol AVL

In [ ]:
class Node:
  def __init__(self, nombre):
    self.nombre = nombre
    self.izq = None
    self.der = None
    self.padre = None
    self.altura = 1

class AVL:
    def __init__(self):
        self.raiz = None

    # REEQUILIBRIO
    def altura (self, nodo):
        if nodo is None:
            return 0
        return max(self.altura(nodo.izq), self.altura(nodo.der)) + 1


    def factor (self, nodo):
        if nodo is None:
            return 0
        return  self.altura(nodo.der) - self.altura(nodo.izq)

    # Rotaciones
    def rotar_derecha(self, nodo):
        nodo_izquierdo = nodo.izq
        T2 = nodo_izquierdo.der

        nodo_izquierdo.der = nodo
        nodo.izq= T2

        nodo.altura = 1 + max(self.altura(nodo.izq), self.altura(nodo.der))
        nodo_izquierdo.altura = 1 + max(self.altura(nodo_izquierdo.izq), self.altura(nodo_izquierdo.der))
        return nodo_izquierdo

    def rotar_izquierda(self, nodo):
        nodo_derecho = nodo.der
        if nodo_derecho is None:
            return nodo

        T3 = nodo_derecho.izq

        nodo_derecho.izq = nodo
        nodo.der = T3

        nodo.altura = 1 + max(self.altura(nodo.izq), self.altura(nodo.der))
        nodo_derecho.altura = 1 + max(self.altura(nodo_derecho.izq), self.altura(nodo_derecho.der))
        return nodo_derecho

    def insertar(self, nombre):
        self.raiz = self._insertar_recur(self.raiz, nombre)
        print("Inorden:", end=" ")
        self.inorden(self.raiz)
        print()
        self.generar_dot_arbol('arbol.dot')
        #self.generar_imagen_arbol('arbol.dot', 'arbol.png')

    def _insertar_recur(self, raiz, nombre, padre = None):
        if raiz is None:
            return Node(nombre)

        if nombre < raiz.nombre:
            if raiz.izq is None:
                raiz.izq = Node(nombre)
                raiz.izq.padre = raiz
            else:
                raiz.izq = self._insertar_recur(raiz.izq, nombre, raiz)
        else:
            if raiz.der is None:
                raiz.der = Node(nombre)
                raiz.der.padre = raiz
            else:
                raiz.der = self._insertar_recur(raiz.der, nombre, raiz)

        balance = self.factor(raiz)

        if balance > 1 and self.factor(raiz.izq) >= 0:
            return self.rotar_derecha(raiz)
        if balance < -1 and self.factor(raiz.der) <= 0:
            return self.rotar_izquierda(raiz)
        if balance > 1 and self.factor(raiz.izq) < 0:
            raiz.izq = self.rotar_izquierda(raiz.izq)
            return self.rotar_derecha(raiz)
        if balance < -1 and self.factor(raiz.der) > 0:
            raiz.der = self.rotar_derecha(raiz.der)
            return self.rotar_izquierda(raiz)

        return raiz

    def inorden(self, raiz):
        if raiz:
            self.inorden(raiz.izq)
            print(raiz.nombre)
            self.inorden(raiz.der)

    def vaciar_arbol(self):
        self.raiz = None

    def generar_dot_arbol(self, filename):
        # Inicializar un nuevo gráfico de Graphviz
        dot = Digraph()

        # Función auxiliar para recorrer el árbol y agregar nodos y aristas al gráfico
        def agregar_nodos_y_aristas(nodo):
            if nodo is not None:
                # Agregar el nodo actual al gráfico
                dot.node(str(nodo.nombre), label=str(nodo.nombre))

                # Recorrer los nodos hijos izquierdo y derecho
                if nodo.izq is not None:
                    dot.edge(str(nodo.nombre), str(nodo.izq.nombre))
                    agregar_nodos_y_aristas(nodo.izq)
                if nodo.der is not None:
                    dot.edge(str(nodo.nombre), str(nodo.der.nombre))
                    agregar_nodos_y_aristas(nodo.der)

        # Agregar nodos y aristas al gráfico
        agregar_nodos_y_aristas(self.raiz)

        # Escribir el gráfico en un archivo DOT
        dot.render(filename, format='png')

    def buscar(self, nombre):
        return self._buscar_recur(self.raiz, nombre)

    def _buscar_recur(self, nodo, nombre):
        if nodo is None:
            return None
        elif nombre == nodo.nombre:
            return nodo
        elif nombre < nodo.nombre:
            return self._buscar_recur(nodo.izq, nombre)
        else:
            return self._buscar_recur(nodo.der, nombre)

    def nivel(self, nodo):
        if nodo:
            return self._obtener_nivel_recur(self.raiz, nodo, 1)
        else:
            return None

    def _obtener_nivel_recur(self, raiz, nodo, nivel_actual):
        if raiz is None:
            return None
        if raiz == nodo:
            return nivel_actual
        nivel_izq = self._obtener_nivel_recur(raiz.izq, nodo, nivel_actual + 1)
        if nivel_izq is not None:
            return nivel_izq
        nivel_der = self._obtener_nivel_recur(raiz.der, nodo, nivel_actual + 1)
        return nivel_der

    def obtener_padre(self, nodo):
        if nodo:
            return self._obtener_padre_recur(self.raiz, nodo, None)
        else:
            return None

    def _obtener_padre_recur(self, raiz, nodo, padre_actual):
        if raiz is None:
            return None
        elif raiz.izq == nodo or raiz.der == nodo:
            return raiz
        elif nodo.nombre < raiz.nombre:
            return self._obtener_padre_recur(raiz.izq, nodo, raiz)
        else:
            return self._obtener_padre_recur(raiz.der, nodo, raiz)

    def obtener_abuelo(self, nodo):
        if nodo and nodo.padre:
            return nodo.padre.padre.nombre if nodo.padre.padre else None
        return None

    def obtener_tio(self, nodo):
        if nodo and nodo.padre and nodo.padre.padre:
            if nodo.padre == nodo.padre.padre.izq:
                return nodo.padre.padre.der.nombre if nodo.padre.padre.der else None
            else:
                return nodo.padre.padre.izq.nombre if nodo.padre.padre.izq else None
        return None

In [ ]:
print(dir(AVL))
#print(arbol.raiz.datos)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_buscar_recur', '_insertar_recur', '_obtener_nivel_recur', '_obtener_padre_recur', 'altura', 'buscar', 'factor', 'generar_dot_arbol', 'inorden', 'insertar', 'nivel', 'obtener_abuelo', 'obtener_padre', 'obtener_tio', 'rotar_derecha', 'rotar_izquierda', 'vaciar_arbol']



### Algoritmo Principal (alpha)

In [ ]:
# CREAR ARBOL
arbol = AVL()
raiz = None

# IMAGENES POR CARPETA
destino = "/content"
carpetas_exist = ["bike", "cars", "cats", "dogs", "flowers", "horses", "human"]
organizar_carpetas(destino, carpetas_exist)
imagenes = procesar_imagenes(destino, carpetas_exist)


def separador():
    print("-----------------------------")

def nombre_en_carpetas(nombre, destino):
    categorias = ["dogs", "flowers", "bikes", "humans", "horses", "cats", "cars"]
    for categoria in categorias:
        carpeta = os.path.join(destino, categoria)
        if os.path.exists(os.path.join(carpeta, nombre)):
            return True
    return False



#### op1

In [ ]:
def option_1(arbol, destino):
    print("INSERTAR IMAGEN")
    insertados = []
    separador()

    while True:
        nuevo = input("Ingrese el nombre del nodo a insertar: ")

        if not nuevo:
            raise ValueError("El nombre del archivo no puede ser vacío")
        if not isinstance(nuevo, str):
           raise TypeError("El nombre del archivo debe ser una cadena de caracteres")
        separador()

        if nuevo in insertados:
            print("¡Esta imagen ya ha sido insertada! Por favor, ingrese otro nombre.")
            continue

        if nombre_en_carpetas(nuevo, destino):
            nodo = Node(nuevo)
            arbol.insertar(nuevo)
            insertados.append(nuevo)
            print(f"Imagen '{nuevo}' insertada en el árbol.")

        else:
            print("El archivo no existe en ninguna de las carpetas.")

        while True:
            opcion_continuar = input("¿Desea insertar otra imagen? (si/no): ").lower()
            if opcion_continuar in ["si", "no"]:
                break
            else:
                print("Por favor, ingrese  sí o  no.")

        if opcion_continuar == "no":
            separador()
            break


#### op2


In [ ]:
def option_2():
    return

#### op3


In [ ]:
def option_3():
    print("BUSCAR")
    separador()

    while True:
        if arbol.raiz is None:
            print("El árbol está vacío. No se puede buscar ningún nodo.")
            separador()
            break

        nombre_archivo = input("Ingrese el nombre del nodo a buscar: ")
        if not nombre_archivo:
            raise ValueError("El nombre del archivo no puede ser vacío")
        if not isinstance(nombre_archivo, str):
           raise TypeError("El nombre del archivo debe ser una cadena de caracteres")
        separador()

        nodo_encontrado = arbol.buscar(nombre_archivo)
        if nodo_encontrado:
            nivel = arbol.nivel(nodo_encontrado.nombre)
            if nivel is not None:
                print(f"Nivel del nodo: {nivel}")
            else:
                print(f"No se pudo obtener el nivel del nodo.")

            factor_balance = arbol.factor(nodo_encontrado)
            if factor_balance is not None:
                print(f"Factor de balance del nodo: {factor_balance}")
            else:
                print(f"No se pudo obtener el factor de balance del nodo.")

            padre = arbol.obtener_padre(nodo_encontrado)
            if padre is not None:
                print(f"Padre del nodo: {padre}")
            else:
                print(f"No se pudo obtener el padre del nodo.")

            abuelo = arbol.obtener_abuelo(nodo_encontrado)
            if abuelo is not None:
                print(f"Abuelo del nodo: {abuelo}")
            else:
                print(f"No se pudo obtener el abuelo del nodo.")

            tio = arbol.obtener_tio(nodo_encontrado)
            if tio is not None:
                print(f"Tío del nodo: {tio}")
            else:
                print(f"No se pudo obtener el tío del nodo.")
        else:
            print(f"No se encontró el nodo con nombre '{nombre_archivo}' en el árbol AVL.")

        while True:
            opcion = input("¿Desea buscar otro? ").lower()
            if opcion == 'si' or opcion == 'no':
                break
            else:
                print("Por favor, ingrese 'si' o 'no'.")
        if opcion != 'si':
            separador()
            break


#### op4

In [ ]:
def option_4():
    return

#### op5


In [ ]:
def option_5():
    return

#### main

In [ ]:
# MAIN
print("Bienvenido al algoritmo del Árbol Binario Balanceado.")
separador()
print("Creadores: Juan Aguirre, Daniela Guerra, Maria Gabriela Reyes.")
separador()

#Validar nombre de user
#while True:
 #   user = input("Por favor, ingrese su nombre (debe tener más de 3 letras y no puede ser un número):\n")
  #  if len(user) > 3 and not user.isdigit():
   #     break
    #print("El nombre debe tener más de 3 letras y no puede ser un número. Inténtelo de nuevo.")

while True:
    print("Hola, ¿Que opcion te gustaria utilizar hoy?")
    print("1. Insertar nodo. ")
    print("2. Eliminar Nodo. ")
    print("3. Buscar Nodo.")
    print("4. Buscar Nodo (con parametros especificos).")
    print("5. Mostrar el recorrido por niveles del arbol.")
    print("6. Salir del programa.")

    while True:
        try:
            option = int(input("Seleccione una opción: "))
            if option < 1 or option > 6:
                raise ValueError("La opción debe ser un número del 1 al 6.")
            break
        except ValueError:
            print("Por favor, ingrese un número válido del 1 al 6.")
    separador()

    if option == 1:
        option_1(arbol, destino)

    elif option == 2:
        option_2()

    elif option == 3:
        option_3()

    elif option == 4:
      print("Eleccion escogida: [#4. Buscar Nodo (con parametros especificos).]")

    elif option == 5:
        print("Eleccion escogida: [#5. Mostrar el recorrido por niveles del arbol.]")

    elif option == 6:
        print("Eleccion escogida: [#6. Salir del programa y vaciar el árbol.]")
        arbol.vaciar_arbol()
        print("El árbol ha sido vaciado.")
        print("Hasta luego!")
        arbol.generar_dot_arbol('arbol.dot')

        separador()



Bienvenido al algoritmo del Árbol Binario Balanceado.
-----------------------------
Creadores: Juan Aguirre, Daniela Guerra, Maria Gabriela Reyes.
-----------------------------
Hola, ¿Que opcion te gustaria utilizar hoy?
1. Insertar nodo. 
2. Eliminar Nodo. 
3. Buscar Nodo.
4. Buscar Nodo (con parametros especificos).
5. Mostrar el recorrido por niveles del arbol.
6. Salir del programa.
Seleccione una opción: 3
-----------------------------
BUSCAR
-----------------------------
El árbol está vacío. No se puede buscar ningún nodo.
-----------------------------
Hola, ¿Que opcion te gustaria utilizar hoy?
1. Insertar nodo. 
2. Eliminar Nodo. 
3. Buscar Nodo.
4. Buscar Nodo (con parametros especificos).
5. Mostrar el recorrido por niveles del arbol.
6. Salir del programa.
Seleccione una opción: 1
-----------------------------
INSERTAR IMAGEN
-----------------------------
Ingrese el nombre del nodo a insertar: 0006.png
-----------------------------
Inorden: 0006.png 
Archivo DOT generado: ar

KeyboardInterrupt: Interrupted by user

In [ ]:
def menu_3(arbol, archivo):
    op = 0
    while op != 6:
        print("¿Que accion le gustaria hacer con:", archivo.nombre, "?")
        separador()
        print("1. Buscar nivel del nodo. ")
        print("2. Averiguar el factor de balanceo del nodo. ")
        print("3. Buscar el padre del nodo.")
        print("4. Buscar el abuelo del nodo.")
        print("5. Buscar el tio del nodo.")
        print("6. Salir de estas opciones.")
        separador()

        while True:
            try:
                op = int(input("Seleccione una opción: "))
                if op < 1 or option > 6:
                    raise ValueError("La opción debe ser un número del 1 al 6.")
                break
            except ValueError:
                print("Por favor, ingrese un número válido del 1 al 6.")
        separador()
#_____________________
        if op == 1:
            print("Nivel del nodo")
            separador()

            nivel = arbol.buscar_nivel(arbol.raiz, archivo)

            if nivel is not None:
                print(f"El nivel del nodo con nombre de archivo '{archivo}' es: {nivel}")
            else:
                print(f"No se encontró el nivel de '{archivo}'.")
            separador()
#_____________________
        elif op == 2:
            print("Factor de balance del nodo")
            separador()

            nodo = archivo
            if nodo:
                factor = arbol.factor(nodo)
                print(f"El factor del nodo con nombre de archivo '{archivo}' es: {factor}")
            else:
                print(f"No se encontró el factor de '{archivo}'.")
            separador()
#_____________________
        elif op == 3:
            print("Padre del nodo")
            separador()

            padre = arbol.obtener_padre(arbol.raiz, archivo)

            if padre is not None:
                print(f"El padre del nodo con nombre de archivo '{archivo}' es: {padre}")
            else:
                print(f"No se encontró padre del nodo con nombre de archivo '{archivo}'.")
            separador()
#_____________________
        elif op == 4:
            print("Abuelo del nodo")
            separador()

            abuelo = arbol.obtener_abuelo(arbol.raiz, archivo)

            if abuelo is not None:
                print(f"El abuelodel nodo con nombre de archivo '{archivo}' es: {abuelo}")
            else:
                print(f"No se encontró el abuelo del nodo con nombre de archivo '{archivo}'.")
            separador()
#_____________________
        elif op == 5:
            print("Tio del nodo")
            separador()

            tio = arbol.obtener_tio(arbol.raiz, archivo)

            if tio is not None:
                print(f"El tio del nodo con nombre de archivo '{archivo}' es: {tio}")
            else:
                print(f"No se encontró el tio del nodo con nombre de archivo '{archivo}'.")
            separador()
#_____________________
        elif op == 6:
            print("Volver al menú principal")
            break